# Pre-Process Ho-Hsian's Data

In [29]:
import numpy as np
from pathlib import Path
PATH = Path('/ds/hohsiangwu/projects/semantic_search')

In [403]:
import torch
torch.__version__

'0.3.1.post2'

In [402]:
! ls /ds/hohsiangwu/projects/semantic_search

Python_Code_Search.ipynb	     py_func_sum.epoch01-val2.21853.hdf5
__pycache__			     py_func_sum.epoch02-val1.91378.hdf5
codeSearch_Model.hdf5		     py_func_sum.epoch02-val1.97722.hdf5
codeSearch_Model_20.hdf5	     py_func_sum.epoch02-val2.13045.hdf5
core				     py_func_sum.epoch02-val2.13085.hdf5
cs_model_.epoch01-val-0.77763.hdf5   py_func_sum.epoch03-val1.93079.hdf5
cs_model_.epoch01-val-0.77858.hdf5   py_func_sum.epoch03-val1.97410.hdf5
cs_model_.epoch01-val-0.77885.hdf5   py_func_sum.epoch03-val2.08514.hdf5
cs_model_.epoch01-val-0.77928.hdf5   py_func_sum.epoch03-val2.08926.hdf5
cs_model_.epoch02-val-0.78204.hdf5   py_func_sum.epoch04-val1.91135.hdf5
cs_model_.epoch02-val-0.78222.hdf5   py_func_sum.epoch04-val1.92982.hdf5
cs_model_.epoch02-val-0.78283.hdf5   py_func_sum.epoch04-val1.97054.hdf5
cs_model_.epoch02-val-0.78312.hdf5   py_func_sum.epoch04-val2.05687.hdf5
cs_model_.epoch03-val-0.78547.hdf5   py_func_sum.epoch04-val2.06309.hdf5
cs_model_.epoch03-val-0.78561.hdf5   py_fun

In [30]:
with open(PATH/'train.function', 'r') as f:
    t_code = f.readlines()

with open(PATH/'valid.function', 'r') as f:
    v_code = f.readlines()

with open(PATH/'test.function', 'r') as f:
    holdout_code = f.readlines()

train_code = t_code + v_code

print(f'train_code rows: {len(train_code):,}')
print(f'holdout_code rows: {len(holdout_code):,}')
print(f'total code rows: {len(holdout_code + train_code):,}')


with open(PATH/'train.docstring', 'r') as f:
    t_comment = f.readlines()

with open(PATH/'valid.docstring', 'r') as f:
    v_comment = f.readlines()

with open(PATH/'test.docstring', 'r') as f:
    holdout_comment = f.readlines()

train_comment = t_comment + v_comment

print(f'\ntrain_comment rows: {len(train_comment):,}')
print(f'holdout_comment rows: {len(holdout_comment):,}')
print(f'total comment rows: {len(holdout_comment + train_comment):,}')

train_code rows: 4,978,625
holdout_code rows: 50,290
total code rows: 5,028,915

train_comment rows: 4,978,625
holdout_comment rows: 50,290
total comment rows: 5,028,915


In [15]:
from ktext.preprocess import processor
code_proc = processor(hueristic_pct_padding=.7, keep_n=20000)
t_code = code_proc.fit_transform(train_code)

comment_proc = processor(append_indicators=True, hueristic_pct_padding=.7, keep_n=15000, padding ='post')
t_comment = comment_proc.fit_transform(train_comment)

 See full histogram by insepecting the `document_length_stats` attribute.
 See full histogram by insepecting the `document_length_stats` attribute.


In [16]:
from pathlib import Path
PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery/')

In [17]:
import dill as dpickle
import numpy as np

# Save the preprocessor
with open(PATH/'py_code_proc.dpkl', 'wb') as f:
    dpickle.dump(code_proc, f)

with open(PATH/'py_comment_proc.dpkl', 'wb') as f:
    dpickle.dump(comment_proc, f)

# Save the processed data
np.save(PATH/'py_t_code_vecs.npy', t_code)
np.save(PATH/'py_t_comment_vecs.npy', t_comment)

# Get TF USE Embeddings

In [11]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")

with tf.Session() as session:
    def get_embeddings(text_blob_list):
            emb = session.run(embed(text_blob_list))
            return emb

from more_itertools import chunked
train_chunked = list(chunked(train_comment, 500000))

INFO:tensorflow:Initialize variable module_2/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_0


INFO:tensorflow:Initialize variable module_2/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_0


INFO:tensorflow:Initialize variable module_2/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_1


INFO:tensorflow:Initialize variable module_2/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_1


INFO:tensorflow:Initialize variable module_2/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_10


...

INFO:tensorflow:Initialize variable module_2/SNLI/Classifier/tanh_layer_0/bias:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/bias


INFO:tensorflow:Initialize variable module_2/SNLI/Classifier/tanh_layer_0/weights:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/weights


INFO:tensorflow:Initialize variable module_2/SNLI/Classifier/tanh_layer_0/weights:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/weights


INFO:tensorflow:Initialize variable module_2/global_step:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with global_step


INFO:tensorflow:Initialize variable module_2/global_step:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with global_step


In [12]:
len(train_chunked)

10

In [13]:
i = 1
use_emb_list = []

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    for x in train_chunked:
        print(i)
        i+=1
        #75th percentile of comment length is 89 characters, so I made limit 200.
        use_emb_list.append(get_embeddings([s[:200] for s in x]))

1
2
3
4
5
6
7
8
9
10


In [14]:
import numpy as np
from pathlib import Path
PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery/')
use_emb = np.concatenate(use_emb_list)

#save the embedding !
np.save(PATH/'use_emb.npy', use_emb)

In [18]:
use_emb.shape

(4978625, 512)

# Organize Fastai Language Model Embeddings

In [21]:
import numpy as np


In [22]:
val_avg_emb = np.load(PATH/'val_avg_emb.npy')

In [23]:
train_avg_emb = np.load(PATH/'train_avg_emb.npy')

In [24]:
train_avg_emb.shape

(4777470, 400)

In [25]:
val_avg_emb.shape

(201155, 400)

In [26]:
concat_train_avg_emb = np.concatenate([train_avg_emb, val_avg_emb])

In [27]:
concat_train_avg_emb.shape

(4978625, 400)

In [28]:
np.save(PATH/'concat_train_avg_emb.npy', concat_train_avg_emb)

In [29]:
val_max_emb = np.load(PATH/'val_max_emb.npy')
train_max_emb = np.load(PATH/'train_max_emb.npy')

val_last_emb = np.load(PATH/'val_last_emb.npy')
train_last_emb = np.load(PATH/'train_last_emb.npy')

concat_train_max_emb = np.concatenate([train_max_emb, val_max_emb])
concat_train_last_emb = np.concatenate([train_last_emb, val_last_emb])

In [30]:
np.save(PATH/'concat_train_max_emb.npy', concat_train_max_emb)
np.save(PATH/'concat_train_last_emb.npy', concat_train_last_emb)

In [31]:
concat_train_max_emb.shape, concat_train_last_emb.shape

((4978625, 400), (4978625, 400))

In [32]:
combined_fastailm_emb = np.concatenate([concat_train_avg_emb, concat_train_max_emb, concat_train_last_emb], axis=1)

In [35]:
combined_fastailm_emb.shape

(4978625, 1200)

In [36]:
np.save(PATH/'combined_fastailm_emb.npy', combined_fastailm_emb)

# File Locations

### Important Files in `/ds/hohsiangwu/projects/semantic_search`

1. `{train, valid, test}.function`:   text file, each line is a tokenized function 

         - train + valid function rows: 4,978,625
         - test function rows: 50,290
         - total function rows: 5,028,915


2. `{train, valid, test}.docstring`:  text file, each line is a tokenized docstring

         - train + valid comment rows: 4,978,625
         - test comment rows: 50,290
         - total comment rows: 5,028,915


### Important Files in `/ds/hamel/CodeML/Get_Python_From_BigQuery/`

1.  `use_emb.npy`            :          Google Universal Sentence Encoder.  shape: (4978625, 512)

2. `concat_train_avg_emb.npy`:          language model average pooling.     shape: (4978625, 400)

3. `concat_train_max_emb.npy`:          language model max pooling.         shape: (4978625, 400)

4. `concat_train_last_emb.npy`:         language model last hidden state.   shape: (4978625, 400)

5. `combined_fastailm_emb`    :         Horizontal concat of [2, 3, 4].     shape: (4978625, 1200)

6.  `codeSearch_Model_frozen.hdf5`:     My best keras model with val_loss = -0.8061 cosine proximity loss.   

```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   

Encoder-Input (InputLayer)   (None, 55)                0         
_________________________________________________________________
Encoder-Model (Model)        (None, 800)               19847200  
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              820224    
_________________________________________________________________
bn-1 (BatchNormalization)    (None, 1024)              4096      
_________________________________________________________________
dense_4 (Dense)              (None, 1200)              1230000   


Total params: 21,901,520
Trainable params: 2,052,272
Non-trainable params: 19,849,248
```
7. `lm_fastai_codecomment_model.pytorch`:  This is the language model trained with fastai, you will have to have the latest version of the fast.ai library

8.  `lm_fastai_codecomment_model_state_dict`: This is the state dict of the language model, a more lightweight way of re-instantiating the model's parameters.

9.  `fitlam_index.nmslib`:  This is the nmslib index that has all the code after it has been vectorized by the language model.  

### Important Notebooks

1.  `hamel/CodeML/Get_Python_From_BigQuery/Parse_Ho_Hsian_Files.ipynb`  this notebook where I do things that are CPU intense:
 - preprocess the {training, validation, lineage} files. 
 - run all the comments through the vectorizers (did a version for both Google and My own languagel model).
 - Loaded all the vectors into an NMS Lib
 - **This is the notebook where the actual demo lives**

2. `hamel/fastai/courses/dl1/lang-model-code-comments.ipynb`:  this notebook is where i trained the fastai language model, and also where I then used the trained model to vectorize all the comments.

3. `/hamel/CodeML/projects/function_summarizer/keras-code-search.ipynb` this is the notebook where I
 - train a function summarizer 
 - fine tune the function summarizer to predict embedding instead of docstring
 - make predictions for all the training data to vectorize all the code (happens on GPU).  This is the code that is loaded into the index where you want to do nearest neighbor search from.

4. `hamel/CodeML/Get_Python_From_BigQuery/Get%20Data%20For%20Python_Code_Search.ipynb` - this notebook I was using to build my own training set, but I abandoned this in favor of Ho-Hsiang's Training dataset which he gave me.

## Place Embeddings Into NMSLIB Index

In [37]:
import nmslib

In [52]:
google_use_index = nmslib.init(method='hnsw', space='cosinesimil')

In [53]:
google_use_index.loadIndex(f'{strPATH}/google_use_index.nmslib')

In [39]:
# google_use_index.addDataPointBatch(use_emb)
# google_use_index.createIndex({'post': 2}, print_progress=True)

#### Test Google's Index

In [51]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/1'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/1'.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/var

In [55]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    test = session.run(embed(['change the color of the page']))

In [58]:
ids, _ = google_use_index.knnQuery(test, k=1)

In [64]:
train_comment[ids[0]]

'change the color of the text on the canvas\n'

In [67]:
strPATH = str(PATH)
print(strPATH)

/ds/hamel/CodeML/Get_Python_From_BigQuery


In [68]:
#google_use_index.saveIndex(f'{strPATH}/google_use_index.nmslib')

## Build Index For Fitlam

In [69]:
# fitlam_index = nmslib.init(method='hnsw', space='cosinesimil')
# fitlam_index.addDataPointBatch(combined_fastailm_emb)
# fitlam_index.createIndex({'post': 2}, print_progress=True)

In [71]:
#fitlam_index.saveIndex(f'{strPATH}/fitlam_index.nmslib')

In [1]:
from pathlib import Path 
PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery/')
strPATH = str(PATH)
print(strPATH)

/ds/hamel/CodeML/Get_Python_From_BigQuery


In [2]:
import nmslib
fitlam_index = nmslib.init(method='hnsw', space='cosinesimil')
fitlam_index.loadIndex(f'{strPATH}/fitlam_index.nmslib')

In [99]:
from fastai.text import *

### Load LangModel Ingredients

PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery/')
from fastai.text import *

trn_lm = np.load(PATH/'trn_lm.npy')
val_lm = np.load(PATH/'val_lm.npy')

with open(PATH/'lm_itos_dict.pkl', 'rb') as f:
    itos = pickle.load(f)

stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

PATH=Path('/ds/hamel/fastai/courses/dl1/code_comment_lm')
#PATH.mkdir(exist_ok=True)

em_sz,nh,nl = 400,400,3
wd=1e-7
bptt=20
bs=32
vs=len(itos)
trn_dl = LanguageModelLoader(trn_lm, bs, bptt)
val_dl = LanguageModelLoader(val_lm, bs, bptt)
md = LanguageModelData(PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7
learner= md.get_model(opt_fn, em_sz, nh, nl, 
                        dropouti=drops[0], 
                        dropout=drops[1], 
                        wdrop=drops[2], 
                        dropoute=drops[3], 
                        dropouth=drops[4])

learner.load('code_comment_lm_v3.fai')

In [240]:
# Load Model From Disk

PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery/')
lang_model_state_dict = torch.load(PATH/'lm_fastai_codecomment_model_state_dict.pytorch')

lang_model = learner.model.cpu()

lang_model.reset()
lang_model.eval()
lang_model.load_state_dict(lang_model_state_dict)

In [241]:
def str2arr(inp):
    """Convert string to array of dimension (seq_len, 1)."""
    arr = np.expand_dims(np.array([stoi[x] for x in inp.lower().split()]), -1)
    return V(T(arr))

def str2emb(inp):
    """Convert string to embedding with lang model"""
    v_arr = str2arr(inp)
    lang_model.reset()
    hidden_states = lang_model(v_arr)[-1][-1]
    lang_model.reset()
    return torch.cat([hidden_states.mean(0), 
                      hidden_states.max(0)[0], 
                      hidden_states[-1]], 
                     -1).data.numpy()

# Battle of the sentence encoders

In [132]:
query = 'tokenize text in parallel and add padding using process based threading'

### 1. Customized Language Model

In [133]:
test_query = str2emb(query)

In [134]:
ids, distances = fitlam_index.knnQuery(test_query, k=1)
print(f'Nearest neighbor comment in vector-space to query: "{query}" \n\n', train_comment[ids[0]])

Nearest neighbor comment in vector-space to query: "tokenize text in parallel and add padding using process based threading" 

 take input text and return tokens w/ part of speech tags using nltk



### 2. Google Universal Sentence Encoder

In [135]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    test_query = session.run(embed([query]))
    
ids, _ = google_use_index.knnQuery(test_query, k=1)

In [136]:
print(f'Nearest neighbor comment in vector-space to query: "{query}" \n\n', train_comment[ids[0]])

Nearest neighbor comment in vector-space to query: "tokenize text in parallel and add padding using process based threading" 

 wrapper around optimizers to apply gradient processors .



In [93]:
!ls /ds/hamel/fastai/fastai

CODE-OF-CONDUCT.md  README.rst		       environment-old.yml  setup.py
LICENSE		    courses		       environment.yml	    tests
MANIFEST	    docs		       fastai		    tutorials
MANIFEST.in	    environment-cpu.yml        requirements.txt
README.md	    environment-nopytorch.yml  setup.cfg


In [95]:
! ln -s /ds/hamel/fastai/fastai fastai

# Code Search Demo

In [404]:
from pathlib import Path
PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery')
train_code_to_lang_vecs = np.load(PATH/'train_code_to_lang_vecs.npy')
train_code_to_lang_vecs.shape

(4978625, 1200)

In [406]:
import nmslib
demo_index = nmslib.init(method='hnsw', space='cosinesimil')
PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery/')
strPATH = str(PATH)
demo_index.loadIndex(f'{strPATH}/demo_index.nmslib')

# demo_index.addDataPointBatch(train_code_to_lang_vecs)
# demo_index.createIndex({'post': 2}, print_progress=True)

In [140]:
from pathlib import Path 
# PATH = Path('/ds/hamel/CodeML/Get_Python_From_BigQuery/')
# strPATH = str(PATH)
# demo_index.saveIndex(f'{strPATH}/demo_index.nmslib')

In [141]:
PATH_2 = Path('/ds/hohsiangwu/projects/semantic_search')
with open(PATH_2/'train.lineage', 'r') as f:
    train_lineage = f.readlines()

In [287]:
import requests
import regex

def print_raw_text(url, ln=0):
    
    raw_url = url.strip()
    linenum = int(regex.search(r'\#L([0-9]*)$', raw_url).group(1))
    filename = regex.search(r'https://github.com/(.*)#L', raw_url).group(1)
    filename = regex.sub(r'/blob/', '/', filename)
    raw_url = 'https://raw.githubusercontent.com/'+filename
    response = requests.get(raw_url)
    assert response.status_code == 200
    html = response.text 
    
    # Print all the things
    print('\n URL:', url, '\n------- Code Preview ----------\n')
    print('\n'.join(html.split('\n')[linenum-1:linenum+20]))
 

In [374]:
def get_first_valid_url(ids):
    for id_ in ids:
        url = train_lineage[id_].strip()
        response = requests.get(url).status_code
        if response == 200:
            return url
    else:
        raise ValueError(f'No valid URLS: {[train_lineage[x] for x in ids]}')

In [361]:
query = 'stream data from kafka into an array'
test_query = str2emb(query)
ids, distances = demo_index.knnQuery(test_query, k=3)

In [362]:
ids

array([3880852,   22183, 2136290], dtype=int32)

In [377]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)
@register_cell_magic
def search(line, cell):
    test_query = str2emb(cell)
    ids, distances = demo_index.knnQuery(test_query, k=5)
    url = get_first_valid_url(ids)
    print_raw_text(url)
    #print_raw_text(train_lineage[ids[0]])

# Semantic Code Search Demo

Results are live, and are retrieved through machine learning technique described here:
https://www.youtube.com/watch?v=nSQIyqtWroU

In [401]:
%%search

keras RNN


 URL: https://github.com/Kismuz/btgym/blob/master/btgym/algorithms/nn/networks.py#L152 
------- Code Preview ----------

def dense_aac_network(x, ac_space, name='dense_aac', linear_layer_ref=noisy_linear, reuse=False):
    """
    Stage3 network: from LSTM flattened output to advantage actor-critic.

    Returns:
        logits tensor
        value function tensor
        action sampling function.
    """
    with tf.variable_scope(name, reuse=reuse):
        # Center-logits:
        logits = norm_layer(
            linear_layer_ref(
                x=x,
                size=ac_space,
                name='action',
                initializer=normalized_columns_initializer(0.01),
                reuse=reuse
            ),
            center=True,
            scale=False,


In [373]:
t.status_code

200